In [ ]:
from selenium import webdriver
from lxml import etree
import time
import itertools
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import re
import math

In [ ]:
driver = webdriver.Chrome(executable_path=r'D:/chromedriver_win32/chromedriver')

In [ ]:
hotel_names = []
review_scores = []
nightly_prices = []
hotel_links = []
review_nums = []
hotel_latlngs=[]

In [ ]:
url = 'https://www.booking.com/searchresults.html?checkin_month=1&checkin_monthday=31&checkin_year=2020&checkout_month=2&checkout_monthday=1&checkout_year=2020&city=20123908&class_interval=1&clear_ht_id=1&dest_id=20123908&dest_type=city&label_click=undef&percent_htype_hotel=1&raw_dest_type=city&room1=A%2CA&rows=25&sb_price_type=total&shw_aparth=1&slp_r_match=0&srpvid=4cac8ab0f390017c&ssb=empty&ssne=Nashville&top_ufis=1&nflt=oos%3D1%3Bht_id%3D204%3Breview_score%3D60%3B&rsf=&offset=0'
driver.get(url)
html = etree.HTML(driver.page_source)
num_hotel = html.xpath('//div[@class="sr_header "]/h1/text()')
temp = re.findall(r'\d+', num_hotel[0]) 
num_hotel = int(list(map(int, temp))[0])

page = num_hotel/25
page=math.ceil(page)+1

In [ ]:
for i in range (0,page):
    try:
        url = 'https://www.booking.com/searchresults.html?checkin_month=1&checkin_monthday=31&checkin_year=2020&checkout_month=2&checkout_monthday=1&checkout_year=2020&city=20123908&class_interval=1&clear_ht_id=1&dest_id=20123908&dest_type=city&label_click=undef&percent_htype_hotel=1&raw_dest_type=city&room1=A%2CA&rows=25&sb_price_type=total&shw_aparth=1&slp_r_match=0&srpvid=4cac8ab0f390017c&ssb=empty&ssne=Nashville&top_ufis=1&nflt=oos%3D1%3Bht_id%3D204%3Breview_score%3D60%3B&rsf=&offset='+str(25*i)
        driver.get(url)
        time.sleep(2)

        y = 800
        for timer in range(0,12):
            driver.execute_script("window.scrollTo(0, "+str(y)+")")
            y += 1000  
            time.sleep(1)

        html = etree.HTML(driver.page_source) 

        hotel_name = html.xpath('//a[@class="hotel_name_link url"]/span[1]/text()')
        review_score = html.xpath('//div[@class="bui-review-score__badge"]/text()')
        nightly_price = html.xpath('//div[@class="bui-price-display__value prco-inline-block-maker-helper"]/text()')
        hotel_link = html.xpath('//a[@class="hotel_name_link url"]/@href')
        review_num = html.xpath('//div[@class="bui-review-score__text"]/text()')
        hotel_latlng = html.xpath('//a[@class="bui-link"]/@data-coords')

        for name in hotel_name:
                    hotel_names.append(name)
        for score in review_score:
                    review_scores.append(score)
        for price in nightly_price:
                    nightly_prices.append(price)
        for link in hotel_link:
                    hotel_links.append(link)        
        for num in review_num:
                    review_nums.append(num)
        for latlng in hotel_latlng:
                    hotel_latlngs.append(latlng)
    except:
        None

In [ ]:
hotelreview = pd.DataFrame(
    {'hotel_name': hotel_names,
     'review_score': review_scores,
     'nightly_price': nightly_prices,
     'hotel_link' : hotel_links,
     'review_num':review_nums,
     'hotel_latlng' :hotel_latlngs
    })

In [ ]:
hotelreview.to_csv('data/hotelreview_raw.csv',index=False)
##..means go outside the folder

In [ ]:
# print(len(hotel_names))
# print(len(review_scores))
# print(len(nightly_prices))
# print(len(hotel_links))
# print(len(review_nums))
# print(len(hotel_latlngs))

In [ ]:
# hotelreview.head()

## Data Prep

In [ ]:
hotelreview_raw = pd.read_csv('data/hotelreview_raw.csv')
#hotelreview_raw.head()

In [ ]:
hotelreview_raw['hotelname'] = hotelreview_raw['hotel_name'].str.replace(r"\r\n","")
hotelreview_raw['nightly_price_2'] = hotelreview_raw['nightly_price'].str.replace(r"\r\n","")
hotelreview_raw['nightly_advertised_price'] = pd.to_numeric(hotelreview_raw['nightly_price_2'].str.replace(r"$",""))
hotelreview_raw['hotel_link_2'] = hotelreview_raw['hotel_link'].str.replace(r"\r\n","")
hotelreview_raw['link'] = "https://www.booking.com"+hotelreview_raw['hotel_link_2'].str.replace(r"\r\n","")
hotelreview_raw['review_score'] = pd.to_numeric(hotelreview_raw['review_score'])
hotelreview_raw['review_num_2'] = hotelreview_raw['review_num'].str.replace(r"reviews","")
hotelreview_raw['review_num'] = pd.to_numeric(hotelreview_raw['review_num_2'].str.replace(r",",""))
hotelreview_raw[['lng','lat']] = hotelreview_raw['hotel_latlng'].str.split(",",expand=True)
hotelreview_raw['lat_lng']= tuple(hotelreview_raw['lat'] +','+ hotelreview_raw['lng'])

import geopy.distance
coords_nss = (36.1277094, -86.7281905)
hotelreview_raw['radius_dist_to_nss'] = [geopy.distance.vincenty(coords_nss,x).miles for x in hotelreview_raw['lat_lng']]

In [ ]:
hotelreview_clean = hotelreview_raw[['hotelname','review_num','review_score','radius_dist_to_nss','nightly_advertised_price','link','lat','lng']]
hotelreview_clean.to_csv('data/hotelreview_clean.csv',index=False)

In [ ]:
# hotelreview_clean.info()
# hotelreview_clean.head()